## Script II - Pandas

In [1]:
import pandas as pd
import numpy as np
import csv
import ast

### Defining a function to clean the datasets

In [18]:
def safe_literal_eval(x):
    """
    Safely parsing string representations of Python objects (dicts, lists).
    Returns None if parsing fails or input is not a valid string.
    """
    if isinstance(x, str) and (x.startswith('{') or x.startswith('[')):
        try:
            return ast.literal_eval(x)
        except (ValueError, SyntaxError):
            return None
    return None

def parse_video_length(time_str):
    """
    Parses time strings like 'HH:MM:SS' or 'MM:SS' into total minutes.
    Returns NaN for invalid formats.
    """
    if not isinstance(time_str, str) or not time_str:
        return np.nan
    
    parts = time_str.split(':')
    try:
        parts = [int(p) for p in parts]
        if len(parts) == 3:  # HH:MM:SS
            hours, minutes, seconds = parts
            total_minutes = hours * 60 + minutes + seconds / 60
        elif len(parts) == 2:  # MM:SS
            minutes, seconds = parts
            total_minutes = minutes + seconds / 60
        else:
            return np.nan
        return total_minutes
    except (ValueError, TypeError):
        return np.nan

In [19]:
def load_and_process_data(filepath, numeric_cols, date_cols, custom_conversions={}, object_cols_to_parse=[]):
    """
    Main loading function. Handles file reading, standard type conversions,
    and two types of custom parsing:
    1. custom_conversions: A dict mapping a column to a specific function.
    2. object_cols_to_parse: A list of columns to parse with a generic function.
    """
    print(f"🔄 Processing file: {filepath}...")
    
    header, good_lines = [], []
    try:
        with open(filepath, mode='r', encoding='utf-8', newline='') as infile:
            reader = csv.reader(infile)
            header = next(reader)
            for row in reader:
                good_lines.append(row)
    except Exception as e:
        print(f"⚠️ Handled a reading error: {e}")

    if not good_lines:
        print(f"❌ No data loaded from {filepath}.")
        return pd.DataFrame()

    df = pd.DataFrame(good_lines, columns=header)
    
    # Performing specific custom conversions first.
    for col, func in custom_conversions.items():
        if col in df.columns:
            df[col] = df[col].apply(func)

    # Performing generic object parsing on the provided list of columns.
    for col in object_cols_to_parse:
        if col in df.columns:
            df[col] = df[col].apply(safe_literal_eval)

    # Applying standard numeric and date conversions.
    valid_numeric_cols = [col for col in numeric_cols if col in df.columns]
    df[valid_numeric_cols] = df[valid_numeric_cols].apply(pd.to_numeric, errors='coerce')
    
    valid_date_cols = [col for col in date_cols if col in df.columns]
    for col in valid_date_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        
    print(f"✅ Successfully processed {filepath}")
    return df

### Applying the Function to the New Files

In [20]:
# Here, I'm defining the structure and paths for the Facebook posts file.
fb_posts_filepath = 'data/2024_fb_posts_president_scored_anon.csv'

fb_numeric_cols = [
    'Overperforming Score', 'advocacy_msg_type_illuminating', 'issue_msg_type_illuminating', 'attack_msg_type_illuminating',
    'image_msg_type_illuminating', 'cta_msg_type_illuminating', 'engagement_cta_subtype_illuminating',
    'fundraising_cta_subtype_illuminating', 'voting_cta_subtype_illuminating', 'covid_topic_illuminating',
    'economy_topic_illuminating', 'education_topic_illuminating', 'environment_topic_illuminating',
    'foreign_policy_topic_illuminating', 'governance_topic_illuminating', 'health_topic_illuminating',
    'immigration_topic_illuminating', 'lgbtq_issues_topic_illuminating', 'military_topic_illuminating',
    'race_and_ethnicity_topic_illuminating', 'safety_topic_illuminating', 'social_and_cultural_topic_illuminating',
    'technology_and_privacy_topic_illuminating', 'womens_issue_topic_illuminating', 'incivility_illuminating',
    'scam_illuminating', 'freefair_illuminating', 'fraud_illuminating', 'Post Views', 'Total Views',
    'Total Views For All Crossposts', 'Total Interactions', 'Likes', 'Comments', 'Shares',
    'Love', 'Wow', 'Haha', 'Sad', 'Angry', 'Care'
]

fb_date_cols = ['Post Created Date', 'Post Created Time']

# Telling my loader to use the special video length parser for this file.
fb_custom_conversions = {'Video Length': parse_video_length}

# Now I'm calling the main function to get my clean DataFrame.
df_fb_posts = load_and_process_data(
    fb_posts_filepath,
    fb_numeric_cols,
    fb_date_cols,
    fb_custom_conversions
)

# Finally, I'm verifying the result.
if not df_fb_posts.empty:
    print("\n--- Facebook Posts DataFrame Info ---")
    df_fb_posts.info()

🔄 Processing file: data/2024_fb_posts_president_scored_anon.csv...


/tmp/ipykernel_2591/427617821.py:42: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')


✅ Successfully processed data/2024_fb_posts_president_scored_anon.csv

--- Facebook Posts DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19009 entries, 0 to 19008
Data columns (total 56 columns):
 #   Column                                                            Non-Null Count  Dtype         
---  ------                                                            --------------  -----         
 0   Facebook_Id                                                       19009 non-null  object        
 1   post_id                                                           19009 non-null  object        
 2   Page Category                                                     19009 non-null  object        
 3   Page Admin Top Country                                            19009 non-null  object        
 4   Post Created                                                      19009 non-null  object        
 5   Post Created Date                                              

In [21]:
# Checking object columns only to check if there are some numerical columns that we missed to convert from object to numerical columns
df_fb_posts[df_fb_posts.select_dtypes(include=['object']).columns.tolist()]

,Facebook_Id,post_id,Page Category,Page Admin Top Country,Post Created,Type,Video Share Status,Is Video Owner?,Sponsor Id,Sponsor Name,Sponsor Category,illuminating_scored_messageelection_integrity_Truth_illuminating
0,a6cb7db6850459b954f7272e14d770617022639f8847cf...,8570b69695e00d8f06b12398ed525497e1712b5369c6fc...,PERSON,US,2023-09-04 19:31:16 EDT,Photo,,-,,,,
1,a6cb7db6850459b954f7272e14d770617022639f8847cf...,41ec27cecd8af40007a9faf8c3e5c9225bcff0b8d58856...,PERSON,US,2023-09-06 20:00:56 EDT,Photo,,-,,,,
2,a6cb7db6850459b954f7272e14d770617022639f8847cf...,1dcb5e00cd1c8d7ee141922f50f29e59e96328231b6937...,PERSON,US,2023-09-21 09:48:09 EDT,Link,,-,,,,
3,a6cb7db6850459b954f7272e14d770617022639f8847cf...,3e5e0a047865ab02fe0f49c343963239fe7774b63e8ab6...,PERSON,US,2023-09-06 20:01:39 EDT,Photo,,-,,,,
4,7ec2cb4abf8effe3d91de57944c56b938b4f33059a6e33...,b83adc0e8ac0aedd39f55b72d723729e114a83d41fd48b...,POLITICAL_CANDIDATE,US,2023-09-27 20:13:08 EDT,Live Video Complete,crosspost,Yes,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
19004,bfe51c6ac2cab17ba5c85883e76f61398031ed57e4cf62...,26510de409d2abf6bfae0ab250b64da17e79498a5edc73...,,,2024-10-31T18:19:27.000Z,,,,,,,
19005,bfe51c6ac2cab17ba5c85883e76f61398031ed57e4cf62...,a555138e67efba4ea3775fba4d06d3e1de1cde54382b65...,,,2024-10-31T12:17:11.000Z,,,,,,,
19006,bfe51c6ac2cab17ba5c85883e76f61398031ed57e4cf62...,06bcd3986e6d7d02109cf2f706c8d194dae46b5a5cc8f0...,,,2024-10-31T01:24:50.000Z,,,,,,,
19007,bfe51c6ac2cab17ba5c85883e76f61398031ed57e4cf62...,53a14d1059933b5aca9615fde383e3f3e8ac0c95befe21...,,,2024-10-30T21:53:50.000Z,,,,,,,


We can notice that all the numerical columns' datatypes have already been converted from object to numeric. 

In [22]:
df_fb_posts['illuminating_scored_messageelection_integrity_Truth_illuminating'].value_counts()

illuminating_scored_messageelection_integrity_Truth_illuminating
    19009
Name: count, dtype: int64

We can see that the column above has all empty strings. So, it's better to delete this column

In [23]:
del df_fb_posts['illuminating_scored_messageelection_integrity_Truth_illuminating']

### Statistics

#### High-Level Descriptive Statistics
This is my first step to get a broad overview of the entire dataset.

In [24]:
# Getting a quick statistical overview of all my numeric columns.
# This gives me count, mean, standard deviation, min, max, and quartiles.
print("--- Numeric Data Summary (Facebook) ---")
display(df_fb_posts.describe())

# Now I'm doing the same for my text-based (object) columns.
# This shows count, number of unique values, the most frequent value, and its frequency.
print("\n--- Categorical Data Summary (Facebook) ---")
display(df_fb_posts.describe(include=['object']))

--- Numeric Data Summary (Facebook) ---


,Post Created Date,Post Created Time,Total Interactions,Likes,Comments,Shares,Love,Wow,Haha,Sad,...,military_topic_illuminating,race_and_ethnicity_topic_illuminating,safety_topic_illuminating,social_and_cultural_topic_illuminating,technology_and_privacy_topic_illuminating,womens_issue_topic_illuminating,incivility_illuminating,scam_illuminating,freefair_illuminating,fraud_illuminating
count,19009,19009,14398.000000,19009.000000,19009.000000,19009.000000,19009.000000,19009.000000,19009.000000,19009.000000,...,19009.000000,19009.000000,19009.000000,19009.000000,19009.000000,19009.000000,19009.000000,18060.000000,19009.000000,19009.000000
mean,2024-03-14 16:38:25.939292160,2025-07-15 13:59:44.330527744,2210.154813,2377.695407,901.583197,320.538955,413.877321,5.868326,105.719712,10.172182,...,0.005576,0.021569,0.032195,0.061708,0.002052,0.025462,0.127887,0.020210,0.002841,0.008627
min,2023-09-01 00:00:00,2025-07-15 00:00:03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2023-12-19 00:00:00,2025-07-15 10:59:46,47.000000,31.000000,8.000000,3.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2024-03-03 00:00:00,2025-07-15 13:49:19,133.000000,139.000000,48.000000,21.000000,4.000000,1.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2024-06-13 00:00:00,2025-07-15 18:00:12,452.750000,738.000000,354.000000,102.000000,70.000000,3.000000,29.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2024-11-06 00:00:00,2025-07-15 23:59:50,470087.000000,351979.000000,93872.000000,76150.000000,244482.000000,4345.000000,99276.000000,56111.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
std,NaN,NaN,13066.622357,11253.469669,3681.980096,1722.159879,3730.940450,52.946979,942.034248,418.325007,...,0.074468,0.145274,0.176523,0.240630,0.045250,0.157527,0.333972,0.140723,0.053224,0.092485



--- Categorical Data Summary (Facebook) ---


,Facebook_Id,post_id,Page Category,Page Admin Top Country,Post Created,Type,Video Share Status,Is Video Owner?,Sponsor Id,Sponsor Name,Sponsor Category
count,19009,19009,19009,19009,19009,19009,19009,19009,19009,19009,19009
unique,21,19009,7,2,18951,10,4,4,1,1,1
top,32fc18da91029ff09bf74fe9887eace6b5d2145809d583...,8570b69695e00d8f06b12398ed525497e1712b5369c6fc...,PERSON,US,2024-10-30T20:57:00.000Z,Link,,-,,,
freq,9013,1,9453,16280,2,7404,15738,13280,19009,19009,19009


#### 2. Single-Column Analysis (Univariate)

Here, I'm digging into individual columns to understand their characteristics.

In [25]:
# Checking the distribution of 'Page Category' to see which are most common.
print("\n--- Top 10 Most Common Page Categories ---")
page_category_counts = df_fb_posts['Page Category'].value_counts()
display(page_category_counts.head(10))

# Getting the total number of likes, comments, and shares across all posts.
print("\n--- Total Engagement Metrics ---")
total_engagement = df_fb_posts[['Likes', 'Comments', 'Shares']].sum()
display(total_engagement)


--- Top 10 Most Common Page Categories ---


Page Category
PERSON                 9453
ACTOR                  3304
POLITICIAN             2595
                       2472
POLITICAL_CANDIDATE    1161
ENTREPRENEUR             23
YOUTH_ORGANIZATION        1
Name: count, dtype: int64


--- Total Engagement Metrics ---


Likes       45197612
Comments    17138195
Shares       6093125
dtype: int64

#### 3. Grouped Analysis (Bivariate)

This is where I start looking for relationships by grouping the data.

In [26]:
# I am now grouping by 'Page Category' to find the average engagement for each.
# This helps me understand which types of pages are most effective.
print("\n--- Average Total Interactions by Page Category (Top 10) ---")
avg_interactions_by_category = df_fb_posts.groupby('Page Category')['Total Interactions'].mean().sort_values(ascending=False)
display(avg_interactions_by_category.head(10))


--- Average Total Interactions by Page Category (Top 10) ---


Page Category
                       11869.191343
POLITICIAN               309.188761
PERSON                   228.441663
POLITICAL_CANDIDATE      180.121406
ENTREPRENEUR             139.304348
ACTOR                    137.633698
YOUTH_ORGANIZATION         5.000000
Name: Total Interactions, dtype: float64

#### 4. Correlation Analysis

I'm creating a correlation matrix to understand how different numeric metrics relate to each other. For example, do posts with more Likes also tend to have more Comments?

In [27]:
# I'm selecting a few key engagement metrics for my correlation matrix.
correlation_cols = ['Total Interactions', 'Likes', 'Comments', 'Shares', 'Post Views', 'Video Length']
correlation_matrix = df_fb_posts[correlation_cols].corr()

print("\n--- Correlation Matrix of Engagement Metrics ---")
display(correlation_matrix)


--- Correlation Matrix of Engagement Metrics ---


,Total Interactions,Likes,Comments,Shares,Post Views,Video Length
Total Interactions,1.000000,0.991617,0.890565,0.863769,0.181678,0.082226
Likes,0.991617,1.000000,0.823384,0.802497,0.492101,0.414158
Comments,0.890565,0.823384,1.000000,0.742744,0.640531,0.499504
Shares,0.863769,0.802497,0.742744,1.000000,0.490458,0.362192
Post Views,0.181678,0.492101,0.640531,0.490458,1.000000,0.405513
Video Length,0.082226,0.414158,0.499504,0.362192,0.405513,1.000000


#### 5. Basic Time-Series Analysis

I'm analyzing how trends change over time.

In [28]:
# To do this, I first need to set the date column as the DataFrame's index.
# I'm using 'Post Created Date' for this.
df_fb_time_indexed = df_fb_posts.set_index('Post Created Date')

# Now I am resampling my data to see the total number of posts made each day.
# 'D' stands for Daily frequency. I could also use 'W' for weekly or 'M' for monthly.
print("\n--- Daily Post Counts ---")
daily_post_counts = df_fb_time_indexed.resample('D')['post_id'].count()
display(daily_post_counts.head())

# I can also calculate the total interactions per day to see when engagement was highest.
print("\n--- Total Daily Interactions ---")
daily_interactions = df_fb_time_indexed.resample('D')['Total Interactions'].sum()
display(daily_interactions.sort_values(ascending=False).head())


--- Daily Post Counts ---


Post Created Date
2023-09-01    50
2023-09-02    27
2023-09-03    21
2023-09-04    29
2023-09-05    40
Freq: D, Name: post_id, dtype: int64


--- Total Daily Interactions ---


Post Created Date
2024-11-05    1640869.0
2024-10-31    1527032.0
2024-10-30    1480453.0
2024-08-10    1131105.0
2024-11-04    1074107.0
Name: Total Interactions, dtype: float64

### Facebook Ads Dataset

In [31]:
# Defining the file structure and loading the data.
fb_ads_filepath = 'data/2024_fb_ads_president_scored_anon.csv'

fb_ads_numeric_cols = [
    'estimated_audience_size', 'estimated_impressions', 'estimated_spend', 'scam_illuminating',
    'election_integrity_Truth_illuminating', 'advocacy_msg_type_illuminating', 'issue_msg_type_illuminating',
    'attack_msg_type_illuminating', 'image_msg_type_illuminating', 'cta_msg_type_illuminating',
    'engagement_cta_subtype_illuminating', 'fundraising_cta_subtype_illuminating',
    'voting_cta_subtype_illuminating', 'incivility_illuminating', 'freefair_illuminating', 'fraud_illuminating'
]
fb_ads_date_cols = ['ad_creation_time']
fb_ads_object_cols = ['delivery_by_region', 'demographic_distribution', 'publisher_platforms', 'illuminating_mentions']

df_fb_ads = load_and_process_data(
    filepath=fb_ads_filepath, 
    numeric_cols=fb_ads_numeric_cols, 
    date_cols=fb_ads_date_cols, 
    object_cols_to_parse=fb_ads_object_cols  # Using the correct parameter name
)

if not df_fb_ads.empty:
    df_fb_ads['total_spend_regions'] = df_fb_ads['delivery_by_region'].apply(
        lambda d: sum(region.get('spend', 0) for region in d.values()) if isinstance(d, dict) else 0
    )
    df_fb_ads['platform_count'] = df_fb_ads['publisher_platforms'].apply(
        lambda p: len(p) if isinstance(p, list) else 0
    )
    print("✅ Feature engineering for Facebook Ads complete.")
    df_fb_ads.info()

🔄 Processing file: data/2024_fb_ads_president_scored_anon.csv...


✅ Successfully processed data/2024_fb_ads_president_scored_anon.csv
✅ Feature engineering for Facebook Ads complete.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31907 entries, 0 to 31906
Data columns (total 43 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   page_id                                    31907 non-null  object        
 1   ad_id                                      31907 non-null  object        
 2   ad_creation_time                           31907 non-null  datetime64[ns]
 3   bylines                                    31907 non-null  object        
 4   currency                                   31907 non-null  object        
 5   delivery_by_region                         31906 non-null  object        
 6   demographic_distribution                   31906 non-null  object        
 7   estimated_audience_size                    31906 non-null 

#### 1. High-Level Descriptive Statistics


In [32]:
if not df_fb_ads.empty:
    print("\n--- Numeric Data Summary (Facebook Ads) ---")
    display(df_fb_ads.describe())
    print("\n--- Categorical Data Summary (Facebook Ads) ---")
    display(df_fb_ads.describe(include=['object']))


--- Numeric Data Summary (Facebook Ads) ---


,ad_creation_time,estimated_audience_size,estimated_impressions,estimated_spend,scam_illuminating,election_integrity_Truth_illuminating,advocacy_msg_type_illuminating,issue_msg_type_illuminating,attack_msg_type_illuminating,image_msg_type_illuminating,cta_msg_type_illuminating,engagement_cta_subtype_illuminating,fundraising_cta_subtype_illuminating,voting_cta_subtype_illuminating,incivility_illuminating,freefair_illuminating,fraud_illuminating,total_spend_regions,platform_count
count,31907,3.190600e+04,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31906.000000,31907.000000,31907.000000
mean,2024-10-21 07:59:52.779013888,4.559165e+05,71890.150881,1653.129317,0.027769,0.052623,0.574469,0.369210,0.199900,0.166458,0.638501,0.075064,0.204538,0.375447,0.149470,0.007491,0.006676,1643.023663,1.912903
min,2022-08-03 00:00:00,0.000000e+00,499.000000,49.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2024-10-22 00:00:00,3.000000e+04,1499.000000,49.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,49.000000,2.000000
50%,2024-10-26 00:00:00,3.000000e+05,7499.000000,149.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,149.000000,2.000000
75%,2024-10-27 00:00:00,1.000001e+06,42499.000000,949.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,923.000000,2.000000
max,2024-11-05 00:00:00,1.000001e+06,1000000.000000,474999.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,474750.000000,4.000000
std,NaN,4.053128e+05,178430.757529,6465.544929,0.164313,0.223284,0.494431,0.482598,0.399931,0.372497,0.480442,0.263499,0.403370,0.484246,0.356557,0.086226,0.081434,6464.603733,0.286064



--- Categorical Data Summary (Facebook Ads) ---


,page_id,ad_id,bylines,currency,delivery_by_region,demographic_distribution,publisher_platforms,illuminating_scored_message,illuminating_mentions,covid_topic_illuminating,...,governance_topic_illuminating,health_topic_illuminating,immigration_topic_illuminating,lgbtq_issues_topic_illuminating,military_topic_illuminating,race_and_ethnicity_topic_illuminating,safety_topic_illuminating,social_and_cultural_topic_illuminating,technology_and_privacy_topic_illuminating,womens_issue_topic_illuminating
count,31907,31907,31907,31907,31906,31906,31906,31906,31906,31906,...,31906,31906,31906,31906,31906,31906,31906,31906,31906,31906
unique,917,31907,857,2,16282,29823,6,3350,61,2,...,2,2,2,2,2,2,2,2,2,2
top,4d66f5853f0365dba032a87704a634f023d15babde973b...,0ddb025b8544e2d58e6977ad417e742a52522b3e1fc1c9...,HARRIS FOR PRESIDENT,USD,{},{},"[facebook, instagram]",82459467ad44f717f2089911f5210cce1ff025dc0cee03...,[],0,...,0,0,0,0,0,0,0,0,0,0
freq,11412,1,9687,31905,2084,2084,29089,1286,12542,31322,...,31593,29243,31335,31876,31872,31400,30835,28905,31884,30029


#### 2. Single-Column Analysis (Univariate)


In [33]:
if not df_fb_ads.empty:
    print("\n--- Top 10 Ad Bylines (Funding Entities) ---")
    display(df_fb_ads['bylines'].value_counts().head(10))


--- Top 10 Ad Bylines (Funding Entities) ---


bylines
HARRIS FOR PRESIDENT                        9687
HARRIS VICTORY FUND                         4629
DONALD J. TRUMP FOR PRESIDENT 2024, INC.    4323
Working America                             1356
Trump National Committee JFC                1200
RALLY BY RELENTLESS PBC                      633
TRUMP 47 COMMITTEE, INC.                     472
Fetterman for PA                             358
MOVEON.ORG POLITICAL ACTION                  254
I Love My Freedom                            216
Name: count, dtype: int64

#### 3. Grouped Analysis (Bivariate)


In [34]:
if not df_fb_ads.empty:
    print("\n--- Advertiser Summary by Ad Spend and Platform Count (Top 10) ---")
    ad_grouped = df_fb_ads.groupby('bylines').agg(
        total_ads=('ad_id', 'count'),
        total_spend=('total_spend_regions', 'sum'),
        avg_spend_per_ad=('total_spend_regions', 'mean'),
        avg_platform_count=('platform_count', 'mean')
    ).sort_values(by='total_spend', ascending=False)
    display(ad_grouped.head(10))



--- Advertiser Summary by Ad Spend and Platform Count (Top 10) ---


,total_ads,total_spend,avg_spend_per_ad,avg_platform_count
bylines,,,,
HARRIS FOR PRESIDENT,9687,15144929,1563.428203,1.998864
HARRIS VICTORY FUND,4629,6492787,1402.632750,1.977533
"DONALD J. TRUMP FOR PRESIDENT 2024, INC.",4323,4429359,1024.603053,1.955355
Trump National Committee JFC,1200,1975907,1646.589167,1.897500
Working America,1356,1629982,1202.051622,1.998525
NowThis Impact,159,1203529,7569.364780,1.132075
RBG PAC,89,1088572,12231.146067,2.000000
FF PAC,163,955559,5862.325153,2.000000
Hard Asset Heroes,60,952902,15881.700000,2.000000


#### 4. Correlation Analysis


In [35]:
if not df_fb_ads.empty:
    print("\n--- Correlation Matrix for Facebook Ads ---")
    ad_corr_cols = ['estimated_impressions', 'total_spend_regions', 'platform_count', 'attack_msg_type_illuminating']
    ad_correlation_matrix = df_fb_ads[ad_corr_cols].corr()
    display(ad_correlation_matrix)


--- Correlation Matrix for Facebook Ads ---


,estimated_impressions,total_spend_regions,platform_count,attack_msg_type_illuminating
estimated_impressions,1.000000,0.643468,-0.047011,0.071473
total_spend_regions,0.643468,1.000000,-0.011726,0.021643
platform_count,-0.047011,-0.011726,1.000000,0.008532
attack_msg_type_illuminating,0.071473,0.021643,0.008532,1.000000


#### 5. Basic Time-Series Analysis


In [36]:
if not df_fb_ads.empty and 'ad_creation_time' in df_fb_ads.columns:
    print("\n--- Daily Ad Spend Over Time ---")
    ad_time_indexed = df_fb_ads.set_index('ad_creation_time')
    daily_ad_spend = ad_time_indexed.resample('D')['total_spend_regions'].sum()
    display(daily_ad_spend.sort_values(ascending=False).head(10))


--- Daily Ad Spend Over Time ---


ad_creation_time
2024-10-27    11853063
2024-10-28     6520117
2024-10-25     3775076
2024-10-22     3505435
2024-10-23     3238233
2024-10-26     2807960
2024-10-24     2633776
2024-10-21     2518146
2024-10-18     1954581
2024-10-17     1910757
Name: total_spend_regions, dtype: int64

## Twitter Posts

In [37]:
# Defining the file structure and loading the data.
tw_posts_filepath = 'data/2024_tw_posts_president_scored_anon.csv'

tw_numeric_cols = [
    'retweetCount', 'replyCount', 'likeCount', 'quoteCount', 'viewCount', 'bookmarkCount',
    'election_integrity_Truth_illuminating', 'advocacy_msg_type_illuminating', 'issue_msg_type_illuminating',
    'attack_msg_type_illuminating', 'image_msg_type_illuminating', 'cta_msg_type_illuminating',
    'engagement_cta_subtype_illuminating', 'fundraising_cta_subtype_illuminating',
    'voting_cta_subtype_illuminating', 'covid_topic_illuminating', 'economy_topic_illuminating',
    'education_topic_illuminating', 'environment_topic_illuminating', 'foreign_policy_topic_illuminating',
    'governance_topic_illuminating', 'health_topic_illuminating', 'immigration_topic_illuminating',
    'lgbtq_issues_topic_illuminating', 'military_topic_illuminating', 'race_and_ethnicity_topic_illuminating',
    'safety_topic_illuminating', 'social_and_cultural_topic_illuminating', 'technology_and_privacy_topic_illuminating',
    'womens_issue_topic_illuminating', 'incivility_illuminating', 'scam_illuminating',
    'freefair_illuminating', 'fraud_illuminating'
]
tw_date_cols = ['createdAt']

# This dataset doesn't have complex object columns to parse.
df_tw_posts = load_and_process_data(tw_posts_filepath, tw_numeric_cols, tw_date_cols)

🔄 Processing file: data/2024_tw_posts_president_scored_anon.csv...


✅ Successfully processed data/2024_tw_posts_president_scored_anon.csv


### Statistics

#### 1. High-Level Descriptive Statistics

In [38]:
if not df_tw_posts.empty:
    print("\n--- Numeric Data Summary (Twitter Posts) ---")
    display(df_tw_posts.describe())
    print("\n--- Categorical Data Summary (Twitter Posts) ---")
    display(df_tw_posts.describe(include=['object']))


--- Numeric Data Summary (Twitter Posts) ---


,retweetCount,replyCount,likeCount,quoteCount,viewCount,createdAt,bookmarkCount,election_integrity_Truth_illuminating,advocacy_msg_type_illuminating,issue_msg_type_illuminating,...,military_topic_illuminating,race_and_ethnicity_topic_illuminating,safety_topic_illuminating,social_and_cultural_topic_illuminating,technology_and_privacy_topic_illuminating,womens_issue_topic_illuminating,incivility_illuminating,scam_illuminating,freefair_illuminating,fraud_illuminating
count,27304.000000,27304.000000,27304.000000,27304.000000,2.730400e+04,27304,27304.000000,26034.000000,26034.000000,26034.000000,...,26034.000000,26034.000000,26034.000000,26034.000000,26034.000000,26034.000000,26034.000000,26034.000000,27304.000000,27304.000000
mean,1322.055193,1063.785013,6913.692829,128.081563,5.070847e+05,2024-04-14 22:56:47.754944512,136.213522,0.037144,0.563686,0.507682,...,0.010986,0.015403,0.037605,0.051971,0.002036,0.023316,0.178574,0.012368,0.001428,0.002747
min,0.000000,0.000000,0.000000,0.000000,5.000000e+00,2023-09-01 00:30:21,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,84.000000,43.000000,393.000000,5.000000,2.785275e+04,2023-11-29 19:16:36.500000,4.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,333.000000,131.000000,1406.000000,17.000000,7.094200e+04,2024-04-19 20:29:15,21.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1071.000000,501.250000,5010.000000,69.000000,3.036630e+05,2024-09-02 16:02:22.500000,76.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,144615.000000,121270.000000,915221.000000,123320.000000,3.335028e+08,2024-11-04 23:40:21,42693.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
std,3405.004240,3174.981654,21590.307989,1131.533468,3.212174e+06,NaN,712.580294,0.189118,0.495937,0.499951,...,0.104237,0.123151,0.190242,0.221972,0.045075,0.150907,0.383003,0.110526,0.037767,0.052339



--- Categorical Data Summary (Twitter Posts) ---


,id,url,source,lang,isReply,isRetweet,isQuote,isConversationControlled,quoteId,inReplyToId,month_year,illuminating_scored_message
count,27304,27304,27304,27304,27304,27304,27304,27304,27304,27304,27304,27304
unique,27304,27304,14,12,2,1,2,2,3160,3310,15,27136
top,cc46051622b8a9c1b883a3bbf12c640b12ac1cbdc7f48a...,f70a206472e9deaf6e313297c1efb891729ced346a0aeb...,Twitter Web App,en,False,False,False,False,,,2024-10,36cb7d55fcf85362ca03f624c2f574f1f55f89db559b17...
freq,1,1,14930,27281,23930,27304,24064,27296,24017,23959,3586,21


#### 2. Single-Column Analysis (Univariate)

In [39]:
if not df_tw_posts.empty:
    print("\n--- Top 10 Tweet Sources/Clients ---")
    display(df_tw_posts['source'].value_counts().head(10))


--- Top 10 Tweet Sources/Clients ---


source
Twitter Web App            14930
Twitter for iPhone          8494
Sprout Social               2933
Twitter Media Studio         499
Twitter for iPad             266
Periscope                    103
Hootsuite Inc.                47
Loomly                        10
Twitter for Advertisers        7
TweetDeck Web App              7
Name: count, dtype: int64

#### 3. Grouped Analysis (Bivariate)


In [40]:
if not df_tw_posts.empty:
    print("\n--- Engagement Summary by Tweet Source (Top 10 by Like Count) ---")
    tw_grouped = df_tw_posts.groupby('source').agg(
        total_tweets=('id', 'count'),
        total_likes=('likeCount', 'sum'),
        avg_retweets=('retweetCount', 'mean'),
        avg_views=('viewCount', 'mean')
    ).sort_values(by='total_likes', ascending=False)
    display(tw_grouped.head(10))


--- Engagement Summary by Tweet Source (Top 10 by Like Count) ---


,total_tweets,total_likes,avg_retweets,avg_views
source,,,,
Twitter Web App,14930,99579349,1317.109310,4.740312e+05
Sprout Social,2933,53302601,3524.793386,1.501097e+06
Twitter for iPhone,8494,31784204,612.623970,2.456221e+05
Twitter Media Studio,499,3691549,1586.212425,4.725235e+05
Periscope,103,255786,596.728155,3.030904e+05
TweetDeck Web App,7,72523,1911.142857,6.247324e+05
Twitter for iPad,266,59237,65.375940,2.135353e+04
Hootsuite Inc.,47,17899,129.978723,2.906662e+04
Twitter for Advertisers,7,5289,97.000000,5.033500e+04


#### 4. Correlation Analysis


In [41]:
if not df_tw_posts.empty:
    print("\n--- Correlation Matrix for Twitter Posts ---")
    tw_corr_cols = ['retweetCount', 'replyCount', 'likeCount', 'quoteCount', 'viewCount']
    tw_correlation_matrix = df_tw_posts[tw_corr_cols].corr()
    display(tw_correlation_matrix)


--- Correlation Matrix for Twitter Posts ---


,retweetCount,replyCount,likeCount,quoteCount,viewCount
retweetCount,1.000000,0.668277,0.931026,0.512319,0.539109
replyCount,0.668277,1.000000,0.650340,0.562347,0.597222
likeCount,0.931026,0.650340,1.000000,0.518073,0.567284
quoteCount,0.512319,0.562347,0.518073,1.000000,0.894019
viewCount,0.539109,0.597222,0.567284,0.894019,1.000000


#### 5. Basic Time-Series Analysis


In [42]:
if not df_tw_posts.empty and 'createdAt' in df_tw_posts.columns:
    print("\n--- Daily Tweet Volume ---")
    tw_time_indexed = df_tw_posts.set_index('createdAt')
    daily_tweet_count = tw_time_indexed.resample('D')['id'].count()
    display(daily_tweet_count.sort_values(ascending=False).head(10))


--- Daily Tweet Volume ---


createdAt
2024-09-11    256
2024-10-02    232
2024-06-28    211
2024-08-20    202
2024-10-30    188
2023-09-28    178
2023-11-09    172
2024-09-25    171
2024-08-23    166
2024-08-22    163
Name: id, dtype: int64

### Conclusion

This analysis covers three datasets related to the 2024 US presidential election: Facebook Posts, Facebook Ads, and Twitter Posts. Each dataset reveals distinct patterns of activity and engagement strategies across the different platforms.

#### Facebook Posts

The analysis of 19,009 Facebook posts shows that engagement is highly concentrated, with a mean of 2,210 interactions per post but a very large standard deviation (~13,066), indicating that a small number of posts drive the majority of engagement.

    Content and Categories: The most frequent post type is 'Link', and the most common page category is 'PERSON' with 9,453 posts.

    Engagement Drivers: Posts from uncategorized pages ('') generate a significantly higher average of ~11,869 interactions, far surpassing official categories like 'POLITICIAN' (~309). As expected, 'Likes', 'Comments', and 'Shares' are all very strongly correlated with 'Total Interactions'.

    Temporal Trends: User interaction peaked on November 5th, 2024, aligning with the election period.

#### Facebook Ads

The dataset contains 31,907 ads, with ad spending peaking on October 27, 2024.

    Spending Patterns: The average ad spend is ~$1,653, but the high standard deviation (~$6,465) points to vastly different budget strategies among advertisers.

    Top Advertisers: 'HARRIS FOR PRESIDENT' is the most prolific advertiser with 9,687 ads and the top spender with over $15.1M in total. However, other groups like 'Hard Asset Heroes' and 'RBG PAC' employ a different strategy, with a much higher average spend per ad of $15,881 and $12,231 respectively.

    Correlations: There is a moderate positive correlation of 0.64 between an ad's estimated impressions and the total amount spent on it.

#### Twitter Posts

The analysis of 27,304 tweets shows that the 'Twitter Web App' is the most used client, responsible for 14,930 tweets.

    Engagement Dynamics: Tweets from 'Sprout Social' receive the highest average number of retweets (~3,525) and views (~1.5M), despite having fewer total tweets than the web app. This suggests it is used by accounts with high-influence.

    Metric Relationships: There is a very strong positive correlation (0.93) between likeCount and retweetCount. viewCount is most strongly correlated with quoteCount (0.89), suggesting quote-tweets are a major driver of visibility.

    Temporal Trends: The highest volume of daily tweets occurred on September 11, 2024, with 256 posts.